In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score  # Importar validación cruzada
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.preprocessing import LabelEncoder
import joblib
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re


In [ ]:
# Ruta del dataset rebalanceado
dataset_path = "Comentarios_Licores_Rebalanceado.xlsx"


In [ ]:
# 1. Cargar el dataset rebalanceado
print("Cargando el dataset desde", dataset_path)
data = pd.read_excel(dataset_path)


Cargando el dataset desde Comentarios_Licores_Rebalanceado.xlsx


In [ ]:
# 2. Preprocesamiento avanzado
print("Iniciando el preprocesamiento del dataset")
data = data[['Comentario', 'Etiqueta']]  # Seleccionar columnas necesarias  # Seleccionar columnas necesarias

Iniciando el preprocesamiento del dataset


In [ ]:
# Mapear etiquetas a valores numéricos usando LabelEncoder
label_encoder = LabelEncoder()
data['Etiqueta'] = label_encoder.fit_transform(data['Etiqueta'])
print("Clases asignadas:", list(label_encoder.classes_))


Clases asignadas: ['Negativo', 'Neutro', 'Positivo']


In [ ]:
# Verificar distribución de clases
print("Distribución de clases en el conjunto de datos original:")
print(data['Etiqueta'].value_counts())


Distribución de clases en el conjunto de datos original:
Etiqueta
0    773
1    760
2    752
Name: count, dtype: int64


In [ ]:
# 3. Dividir datos en entrenamiento y prueba con estratificación
print("Dividiendo los datos en entrenamiento y prueba")
X_train, X_test, y_train, y_test = train_test_split(
    data['Comentario'], data['Etiqueta'], test_size=0.2, random_state=42, stratify=data['Etiqueta']
)


Dividiendo los datos en entrenamiento y prueba


In [ ]:
# Verificar distribución de clases después de la división
print("Distribución de clases en el conjunto de entrenamiento:")
print(y_train.value_counts())

print("Distribución de clases en el conjunto de prueba:")
print(y_test.value_counts())


Distribución de clases en el conjunto de entrenamiento:
Etiqueta
0    618
1    608
2    602
Name: count, dtype: int64
Distribución de clases en el conjunto de prueba:
Etiqueta
0    155
1    152
2    150
Name: count, dtype: int64


In [ ]:
# 4. Preprocesamiento de texto
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')
stemmer = SnowballStemmer('spanish')
negation_words = ['no', 'nunca', 'jamás', 'ni']


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess_text(text):
    negation_flag = False
    # Convertir a minúsculas
    text = text.lower()
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^a-záéíóúñü]', ' ', text)
    # Tokenizar
    tokens = text.split()
    # Eliminar stopwords y aplicar stemming
    processed_tokens = []
    for i, word in enumerate(tokens):
        if word in negation_words:
            negation_flag = True
        elif negation_flag:
            processed_tokens.append('no_' + word)  # Marcar la negación
        elif word in negation_words:
            processed_tokens.append(word)  # Mantener las palabras negativas
            negation_flag = False
        elif word not in spanish_stopwords:
            processed_tokens.append(stemmer.stem(word))
        # Asegurar que la palabra negativa se mantenga
        elif word in negation_words:
            processed_tokens.append(word)
    tokens = processed_tokens
    # Unir tokens
    text = ' '.join(tokens)
    return text

X_train = X_train.apply(preprocess_text)
X_test = X_test.apply(preprocess_text)


In [ ]:
# 5. Vectorización con TF-IDF
print("Iniciando la vectorización de los comentarios con TF-IDF")
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), min_df=5, max_df=0.7)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print("Vectorización completada. Número de características:", len(vectorizer.get_feature_names_out()))

Iniciando la vectorización de los comentarios con TF-IDF
Vectorización completada. Número de características: 568


In [ ]:
# Ajuste del modelo para mejores resultados con frases ambiguas
# 6. Entrenamiento del modelo
print("Entrenando el modelo de Naive Bayes")
model = ComplementNB(alpha=0.1)
model.fit(X_train_tfidf, y_train)


Entrenando el modelo de Naive Bayes


ComplementNB(alpha=0.1)

In [ ]:
# Guardar el modelo, el vectorizador y el label encoder juntos
componentes = {
    "modelo": model,
    "vectorizador": vectorizer,
    "label_encoder": label_encoder
}
joblib.dump(componentes, "modelo_completo.pkl")



['modelo_completo.pkl']

In [ ]:
# 7. Evaluación del modelo
print("Evaluación con validación cruzada")
cross_val_scores = cross_val_score(model, X_train_tfidf, y_train, cv=5, scoring='accuracy')
print("Puntajes de validación cruzada:", cross_val_scores)
print("Promedio de validación cruzada:", cross_val_scores.mean())


Evaluación con validación cruzada
Puntajes de validación cruzada: [0.98360656 0.99726776 0.99453552 0.9890411  0.9890411 ]
Promedio de validación cruzada: 0.9906984055692792


In [ ]:
print("Evaluando el modelo en el conjunto de prueba")
y_pred = model.predict(X_test_tfidf)


Evaluando el modelo en el conjunto de prueba


In [ ]:
print("Reporte de Clasificación:")
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
precision = accuracy_score(y_test, y_pred)
print("Precisión del modelo:", precision)
print("Exactitud del Modelo en Prueba:", accuracy_score(y_test, y_pred))

Reporte de Clasificación:
              precision    recall  f1-score   support

    Negativo       0.96      0.96      0.96       155
      Neutro       0.97      1.00      0.98       152
    Positivo       0.98      0.95      0.96       150

    accuracy                           0.97       457
   macro avg       0.97      0.97      0.97       457
weighted avg       0.97      0.97      0.97       457

Precisión del modelo: 0.9693654266958425
Exactitud del Modelo en Prueba: 0.9693654266958425


In [ ]:
# Mostrar matriz de confusión en consola
cm = confusion_matrix(y_test, y_pred)
print("Matriz de confusión:")
print(cm)

Matriz de confusión:
[[149   3   3]
 [  0 152   0]
 [  6   2 142]]


In [ ]:
# 8. Predicción de nuevos comentarios
def predecir_nuevos_comentarios(nuevos_comentarios):
    # Cargar el modelo, vectorizador y label encoder juntos
    componentes = joblib.load("modelo_completo.pkl")
    model = componentes["modelo"]
    vectorizer = componentes["vectorizador"]
    label_encoder = componentes["label_encoder"]

    # Preprocesar nuevos comentarios
    nuevos_comentarios_prep = [preprocess_text(comentario) for comentario in nuevos_comentarios]

    nuevos_comentarios_tfidf = vectorizer.transform(nuevos_comentarios_prep)
    predicciones = model.predict(nuevos_comentarios_tfidf)

    # Obtener etiquetas originales
    predicciones_etiquetas = label_encoder.inverse_transform(predicciones)

    # Mostrar resultados
    for comentario, etiqueta in zip(nuevos_comentarios, predicciones_etiquetas):
        print(f"Comentario: {comentario} | Sentimiento: {etiqueta}")

# Ejemplo de nuevos comentarios a predecir
nuevos_comentarios = [
    "El producto llegó a tiempo y en excelentes condiciones, lo recomiendo ampliamente.",
    "No tiene buen sabor, esperaba mucho más por el precio que pagué.",
    "Es simplemente perfecto, ideal para reuniones con amigos o familiares.",
    "El empaque llegó dañado, pero el producto en sí estaba bien.",
    "No es malo, pero tampoco es algo que volvería a comprar.",
    "No cumplió con mis expectativas, no lo recomendaría.",
    "El sabor fue excelente, todos en la fiesta quedaron impresionados.",
    "Esperaba más del producto, no estuvo a la altura de mis expectativas.",
    "Muy buen servicio y el licor de alta calidad, repetiré la compra.",
    "No me gustó para nada, decepcionante y no lo recomendaría."
]

predecir_nuevos_comentarios(nuevos_comentarios)



Comentario: El producto llegó a tiempo y en excelentes condiciones, lo recomiendo ampliamente. | Sentimiento: Positivo
Comentario: No tiene buen sabor, esperaba mucho más por el precio que pagué. | Sentimiento: Negativo
Comentario: Es simplemente perfecto, ideal para reuniones con amigos o familiares. | Sentimiento: Positivo
Comentario: El empaque llegó dañado, pero el producto en sí estaba bien. | Sentimiento: Neutro
Comentario: No es malo, pero tampoco es algo que volvería a comprar. | Sentimiento: Negativo
Comentario: No cumplió con mis expectativas, no lo recomendaría. | Sentimiento: Negativo
Comentario: El sabor fue excelente, todos en la fiesta quedaron impresionados. | Sentimiento: Positivo
Comentario: Esperaba más del producto, no estuvo a la altura de mis expectativas. | Sentimiento: Negativo
Comentario: Muy buen servicio y el licor de alta calidad, repetiré la compra. | Sentimiento: Positivo
Comentario: No me gustó para nada, decepcionante y no lo recomendaría. | Sentimiento: